In [ ]:
# !pip install --upgrade transformers

In [ ]:
# !pip install auto-round 

In [ ]:
# !MAX_JOBS=8 pip install flash-attn --no-build-isolation

In [1]:
import os
import torch
from auto_round import AutoRound
from huggingface_hub import HfApi, create_repo, notebook_login, get_token

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


PyTorch Version: 2.8.0+cu126
CUDA Available: True
CUDA Version: 12.6
GPU Name: NVIDIA RTX 6000 Ada Generation
VRAM: 47.4 GB


In [4]:
MODEL_ID = "Qwen/Qwen3-VL-8B-Instruct"
HF_USER = "Vishva007"
OUTPUT_BASE_DIR = "./AutoRound"

In [5]:
notebook_login()

In [6]:
# "AWQ",            # Best for Nvidia GPUs (vLLM, TGI)
# "GPTQ",           # Good compatibility
# "AutoRound",      # Intel default format (Requires auto-round lib to run)
# "GGUF",           # For llama.cpp / Olama

TARGET_FORMATS = "AWQ"

use_fp16 = TARGET_FORMATS in ["AWQ", "GPTQ"]


# High-End GPU Tuning Parameters (A40/A6000/L40)
TUNING_CONFIG = {
    "group_size": 128,
    "sym": True,
    "iters": 600,           # High accuracy (Production grade)
    "nsamples": 384,         # More calibration data
    "batch_size": 4,        # Faster on 48GB VRAM
    "seqlen": 2048,
    "low_gpu_mem_usage": False,   # Keep on GPU for speed
    "enable_torch_compile": True, # JIT acceleration
    "quant_nontext_module": False, # Keep Vision Tower in BF16 (Crucial for VLM accuracy)
}



In [7]:
def push_to_hub(local_dir, repo_name, token):
    """Creates repo and uploads folder to Hugging Face."""
    full_repo_id = f"{HF_USER}/{repo_name}"
    print(f"\n[Hub] Pushing {local_dir} to {full_repo_id}...")
    
    try:
        api = HfApi()
        create_repo(full_repo_id, repo_type="model", exist_ok=True, private=False, token=token)
        
        api.upload_folder(
            folder_path=local_dir,
            repo_id=full_repo_id,
            repo_type="model",
            token=token
        )
        print(f"[Hub] ✅ Successfully uploaded: https://huggingface.co/{full_repo_id}")
    except Exception as e:
        print(f"[Hub] ❌ Error uploading: {e}")

In [8]:

ar = AutoRound(
        MODEL_ID,
        scheme="W4A16",
        model_dtype="fp16" if use_fp16 else "bf16",
        **TUNING_CONFIG
    )

2026-02-01 14:39:54 INFO autoround.py L158: using MLLM mode for multimodal model.
2026-02-01 14:39:54 WARNING modeling_utils.py L4670: `torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2026-02-01 14:40:01 INFO base.py L391: using torch.float16 for quantization tuning


In [9]:
base_name = MODEL_ID.split("/")[-1]

if TARGET_FORMATS == "AWQ":
    dir_suffix = "W4A16-AWQ"
    format_arg = "auto_awq"
elif TARGET_FORMATS == "GPTQ":
    dir_suffix = "W4A16-GPTQ"
    format_arg = "auto_gptq"
else: # AutoRound
    dir_suffix = "W4A16-AutoRound"
    format_arg = "auto_round"

save_dir = os.path.join(OUTPUT_BASE_DIR, dir_suffix)


ar.quantize_and_save(
    format=format_arg, 
    output_dir=save_dir, 
    inplace=False
)

2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.0.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.0.mlp.linear_fc2 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.1.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.1.mlp.linear_fc2 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.2.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.2.mlp.linear_fc2 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.3.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-02-01 14:40:05 WARNING utils.py L425: model.visual.blocks.3.mlp.linear_fc2 skipped qu

(Qwen3VLForConditionalGeneration(
   (model): Qwen3VLModel(
     (visual): Qwen3VLVisionModel(
       (patch_embed): Qwen3VLVisionPatchEmbed(
         (proj): Conv3d(3, 1152, kernel_size=(2, 16, 16), stride=(2, 16, 16))
       )
       (pos_embed): Embedding(2304, 1152)
       (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
       (blocks): ModuleList(
         (0-26): 27 x Qwen3VLVisionBlock(
           (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
           (norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
           (attn): Qwen3VLVisionAttention(
             (qkv): Linear(in_features=1152, out_features=3456, bias=True)
             (proj): Linear(in_features=1152, out_features=1152, bias=True)
           )
           (mlp): Qwen3VLVisionMLP(
             (linear_fc1): Linear(in_features=1152, out_features=4304, bias=True)
             (linear_fc2): Linear(in_features=4304, out_features=1152, bias=True)
             (act_fn): GELUTanh()
           

In [10]:
hf_token = get_token()

if hf_token:
    push_to_hub(save_dir, f"{base_name}-W4A16-AutoRound-{TARGET_FORMATS}", hf_token)
else:
    print("❌ No HF token found. Skipping upload.")



[Hub] Pushing ./AutoRound/W4A16-AWQ to Vishva007/Qwen3-VL-8B-Instruct-W4A16-AutoRound-AWQ...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[Hub] ✅ Successfully uploaded: https://huggingface.co/Vishva007/Qwen3-VL-8B-Instruct-W4A16-AutoRound-AWQ
